In [38]:
import math
import warnings
import torch
import gpytorch
from gpytorch.kernels import RBFKernel
from matplotlib import pyplot as plt
from utils.kernel import LaplacianKernel

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
init_lengthscale = RBFKernel().lengthscale.item()
init_lengthscale

0.6931471824645996

In [27]:
import torch
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.optim import optimize_acqf

# 1. Basic BO

- 目的関数: sphere function
- 代理モデル: `botorch.models.SingleTaskGP`
- 獲得関数: `botorch.acquisition.analitic.ExpectedImprovement`

In [28]:
def sphere_function(x):
    return torch.sum(x ** 2, 1, keepdim=True)

def generate_initial_data(target_function, shape=(10,1)):
    train_x = torch.rand(shape).double()
    train_y = target_function(train_x)
    # best_val = train_y.max().item()
    best_val = train_y.min().item()
    return train_x, train_y, best_val

def get_next_points(x_train, y_train, best_y, bounds, n_points=1):
    y_train = - y_train

    model = SingleTaskGP(x_train, y_train)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)
    
    acq_func = ExpectedImprovement(model=model, best_f=best_y)

    candidates, _ = optimize_acqf(acq_function=acq_func,
                                  bounds=bounds,
                                  q=n_points,
                                  num_restarts=5,
                                  raw_samples=20)
    
    return candidates


n_runs = 100
dim = 5
target_func = sphere_function
init_x, init_y, best_init_y = generate_initial_data(target_func, shape=(10, dim))
bounds = torch.stack([-torch.ones(dim), torch.ones(dim)]).double()

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for i in range(n_runs):
        print(f'Iter: {i+1}')

        new_candidates = get_next_points(init_x, init_y, best_init_y, bounds, n_points=1)

        print(new_candidates)

        new_results = target_func(new_candidates)
        
        print(f'New candidates: {new_candidates}')           
        init_x = torch.cat([init_x, new_candidates])
        init_y = torch.cat([init_y, new_results])

        # best_init_y = init_y.max().item()
        best_init_y = init_y.min().item()
        print(f'Current best y: {best_init_y}')
        print()

Iter: 1
tensor([[ 0.0493, -0.0145,  0.5995,  0.8421,  0.0095]], dtype=torch.float64)
New candidates: tensor([[ 0.0493, -0.0145,  0.5995,  0.8421,  0.0095]], dtype=torch.float64)
Current best y: 0.7138669150618924

Iter: 2
tensor([[ 0.1820, -0.6817,  0.4706,  0.6290,  0.9277]], dtype=torch.float64)
New candidates: tensor([[ 0.1820, -0.6817,  0.4706,  0.6290,  0.9277]], dtype=torch.float64)
Current best y: 0.7138669150618924

Iter: 3
tensor([[-0.2110, -0.0261,  0.9154,  0.1852,  0.6373]], dtype=torch.float64)
New candidates: tensor([[-0.2110, -0.0261,  0.9154,  0.1852,  0.6373]], dtype=torch.float64)
Current best y: 0.7138669150618924

Iter: 4
tensor([[ 0.3600,  0.2901,  0.4862, -0.0464, -0.1579]], dtype=torch.float64)
New candidates: tensor([[ 0.3600,  0.2901,  0.4862, -0.0464, -0.1579]], dtype=torch.float64)
Current best y: 0.4771702581279137

Iter: 5
tensor([[0.0470, 0.2136, 0.3274, 0.1121, 0.0119]], dtype=torch.float64)
New candidates: tensor([[0.0470, 0.2136, 0.3274, 0.1121, 0.0119]

# 2. BO rbf

- 目的関数: sphere function
- 代理モデル: Exact GP (RBF カーネル)
- 獲得関数: `botorch.acquisition.analitic.ExpectedImprovement`

In [29]:
from botorch.models.gpytorch import GPyTorchModel
from botorch.utils.datasets import SupervisedDataset
from gpytorch.distributions import MultivariateNormal
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP


class SimpleCustomGP(ExactGP, GPyTorchModel):

    _num_outputs = 1  # to inform GPyTorchModel API

    def __init__(self, train_X, train_Y):
        # squeeze output dim before passing train_Y to ExactGP
        super().__init__(train_X, train_Y.squeeze(-1), GaussianLikelihood())
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(
            base_kernel=RBFKernel(ard_num_dims=train_X.shape[-1]),
        )
        self.to(train_X)  # make sure we're on the right device/dtype

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

In [33]:
def sphere_function(x):
    return torch.sum(x ** 2, 1, keepdim=True)

def generate_initial_data(target_function, shape=(10,1)):
    train_x = torch.rand(shape).double()
    train_y = target_function(train_x)
    # best_val = train_y.max().item()
    best_val = train_y.min().item()
    return train_x, train_y, best_val

def get_next_points(x_train, y_train, best_y, bounds, n_points=1):
    y_train = - y_train

    model = SimpleCustomGP(x_train, y_train)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)
    
    acq_func = ExpectedImprovement(model=model, best_f=best_y)

    candidates, _ = optimize_acqf(acq_function=acq_func,
                                  bounds=bounds,
                                  q=n_points,
                                  num_restarts=5,
                                  raw_samples=20)
    
    return candidates


n_runs = 100
dim = 5
target_func = sphere_function
init_x, init_y, best_init_y = generate_initial_data(target_func, shape=(10, dim))
bounds = torch.stack([-torch.ones(dim), torch.ones(dim)]).double()

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for i in range(n_runs):
        print(f'Iter: {i+1}')

        new_candidates = get_next_points(init_x, init_y, best_init_y, bounds, n_points=1)

        print(new_candidates)

        new_results = target_func(new_candidates)
        
        print(f'New candidates: {new_candidates}')           
        init_x = torch.cat([init_x, new_candidates])
        init_y = torch.cat([init_y, new_results])

        # best_init_y = init_y.max().item()
        best_init_y = init_y.min().item()
        print(f'Current best y: {best_init_y}')
        print()

Iter: 1
tensor([[ 0.0188, -0.9695,  0.2354, -1.0000, -0.3466]], dtype=torch.float64)
New candidates: tensor([[ 0.0188, -0.9695,  0.2354, -1.0000, -0.3466]], dtype=torch.float64)
Current best y: 0.09332987277060312

Iter: 2
tensor([[-0.5268,  0.2444,  0.4532, -0.7255, -0.0798]], dtype=torch.float64)
New candidates: tensor([[-0.5268,  0.2444,  0.4532, -0.7255, -0.0798]], dtype=torch.float64)
Current best y: 0.09332987277060312

Iter: 3
tensor([[ 0.3714, -0.1850, -0.4345,  0.5051, -0.3159]], dtype=torch.float64)
New candidates: tensor([[ 0.3714, -0.1850, -0.4345,  0.5051, -0.3159]], dtype=torch.float64)
Current best y: 0.09332987277060312

Iter: 4
tensor([[-0.5714, -0.1240,  0.0936,  0.3213,  0.2112]], dtype=torch.float64)
New candidates: tensor([[-0.5714, -0.1240,  0.0936,  0.3213,  0.2112]], dtype=torch.float64)
Current best y: 0.09332987277060312

Iter: 5
tensor([[-0.2370,  0.3616,  0.3430,  0.6788, -0.8161]], dtype=torch.float64)
New candidates: tensor([[-0.2370,  0.3616,  0.3430,  0.

In [34]:
model

SimpleCustomGP(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): RBFKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
)

In [35]:
model.likelihood

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

# 3. BO Laplacian

- 目的関数: sphere function
- 代理モデル: Exact GP (Laplacian カーネル)
- 獲得関数: `botorch.acquisition.analitic.ExpectedImprovement`

In [39]:
from botorch.models.gpytorch import GPyTorchModel
from botorch.utils.datasets import SupervisedDataset
from gpytorch.distributions import MultivariateNormal
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP


class SimpleCustomGP(ExactGP, GPyTorchModel):

    _num_outputs = 1  # to inform GPyTorchModel API

    def __init__(self, train_X, train_Y):
        # squeeze output dim before passing train_Y to ExactGP
        super().__init__(train_X, train_Y.squeeze(-1), GaussianLikelihood())
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(
            base_kernel=LaplacianKernel(ard_num_dims=train_X.shape[-1]),
        )
        self.to(train_X)  # make sure we're on the right device/dtype

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

In [47]:
def sphere_function(x):
    return torch.sum(x ** 2, 1, keepdim=True)

def generate_initial_data(target_function, shape=(10,1)):
    train_x = torch.rand(shape).double()
    train_y = target_function(train_x)
    # best_val = train_y.max().item()
    best_val = train_y.min().item()
    return train_x, train_y, best_val

def get_next_points(x_train, y_train, best_y, bounds, n_points=1):
    y_train = - y_train

    # model = SingleTaskGP(x_train, y_train)
    model = SimpleCustomGP(x_train, y_train)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)
    
    acq_func = ExpectedImprovement(model=model, best_f=best_y)

    candidates, _ = optimize_acqf(acq_function=acq_func,
                                  bounds=bounds,
                                  q=n_points,
                                  num_restarts=5,
                                  raw_samples=20)
    
    return candidates


n_runs = 100
dim = 5
target_func = sphere_function
init_x, init_y, best_init_y = generate_initial_data(target_func, shape=(10, dim))
bounds = torch.stack([-torch.ones(dim), torch.ones(dim)]).double()

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for i in range(n_runs):
        print(f'Iter: {i+1}')

        new_candidates = get_next_points(init_x, init_y, best_init_y, bounds, n_points=1)

        new_results = target_func(new_candidates)
        
        print(f'New candidates: {new_candidates}')
        init_x = torch.cat([init_x, new_candidates])
        init_y = torch.cat([init_y, new_results])

        # best_init_y = init_y.max().item()
        best_init_y = init_y.min().item()
        print(f'Current best y: {best_init_y}')
        print()

Iter: 1
New candidates: tensor([[-0.6172, -0.2208, -0.4988, -1.0000, -0.5797]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 2
New candidates: tensor([[-1.0000, -1.0000, -0.9995,  0.9982,  1.0000]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 3
New candidates: tensor([[-0.3403, -1.0000,  1.0000,  1.0000, -1.0000]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 4
New candidates: tensor([[ 1.0000,  1.0000, -1.0000, -0.3864, -1.0000]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 5
New candidates: tensor([[-1.0000,  1.0000,  1.0000, -0.6412, -1.0000]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 6
New candidates: tensor([[ 0.6835, -1.0000, -0.6772, -0.1746,  0.1351]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 7
New candidates: tensor([[-1.0000, -1.0000,  1.0000, -1.0000,  0.8317]], dtype=torch.float64)
Current best y: 1.1278744939184762

Iter: 8
New candidates: tensor([[-1.0000,

In [44]:
x_train, y_train, _ = generate_initial_data(sphere_function)

model = SimpleCustomGP(x_train, y_train)

In [45]:
model.likelihood

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [46]:
model

SimpleCustomGP(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): LaplacianKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
)